## RPFS Problem (TWCT objective) - Data treatment of result files 

In [ ]:
import pandas as pd
import numpy as np
import os, fnmatch
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
import glob
import os
from pathlib import Path

%matplotlib inline

In [ ]:
import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO

### List files in the output folder 

In [ ]:
rootfolder = os.getcwd()
file_list = glob.glob(os.path.join(rootfolder, 'output') + '/*.csv', recursive=True)
file_list

### Read all the CSV files 

In [ ]:
# Alternative script to treat files with incorrect number of coluns or faulty lines
def alternative_csv_reader(filename, delimiter=',', header=0, names=None):
    with open(filename, 'r') as file:
        lines = file.readlines() 
        count = 1
        line_list = []
        num_columns = 20
        for line in lines:  # Strips the newline character 
            #print("line{}: {}".format(count, line.strip())) 
            nc = len(line.split(','))
            if 'executionId,' in line:
                #num_columns = nc
                print('Detected {0} columns in CSV file.'.format(nc))
            else:
                if 'none,' in line:
                    if nc == num_columns:
                        line_list.append(line)
                    elif nc > num_columns:  # treat strange truncated lines
                        line = line[line.rfind('none,'):]
                        nc = len(line.split(','))
                        if nc == num_columns:
                            print('WARN: truncating line {0}, for having more columns than expected.'.format(count))
                            line_list.append(line)
                        else:
                            print('WARN: Ignoring line {0}, since it has {1} columns, instead of {2}: '.format(count, nc, num_columns), line)
                    else:  # Ignore line
                        print('WARN: Ignoring line {0}: '.format(count), line)    
                elif len(line_list[-1].split(',')) < num_columns:  # current line is a continuation of the previous one
                    line_list[-1] = line_list[-1].replace('\n', '') + line
                    print('*** Treated line {0}: '.format(count), line_list[-1])
                else:  # Ignore line
                    print('WARN: Ignoring line {0}: '.format(count), line)
            count += 1
        # assert all lines have the same number of columns
        count = 1
        for line in line_list:
            nc = len(line.split(','))
            if nc != num_columns:
                print('ERROR: Line {0} has {1} columns, instead of {2}: '.format(count, nc, num_columns), line)
            count += 1
        text_data = StringIO(''.join(line_list))
        #print('line_list: ', str(line_list))
        #print('text_data: ', text_data)
        df = pd.read_csv(text_data, delimiter=delimiter, header=header, names=names)
        return df

### Process all CSV files and append all data to a single dataframe (one per solution method: Wilson, Wagner) 

In [ ]:
%%time

dfdict = dict()
for filepath in file_list:
    print('Processing file ', filepath)
    try:
        df_ = pd.read_csv(filepath, delimiter=',', header=0, names=['executionId','ub_name','instance_name','alpha','n','m','budget_Gamma','wct','permutation','time_spent','time_to_best_sol','mp_total_time','sp_total_time','iterations','num_visited_solutions','num_improvements','is_optimal','validated','gap','lb','cost','wct_validation'])
    except:  # try alternative method to read csv lines
        df_ = alternative_csv_reader(filename, delimiter=',', header=0, names=['executionId','ub_name','instance_name','alpha','n','m','budget_Gamma','wct','permutation','time_spent','time_to_best_sol','mp_total_time','sp_total_time','iterations','num_visited_solutions','num_improvements','is_optimal','validated','gap','lb','cost','wct_validation'])
    filename = filepath[filepath.rfind(os.path.sep)+1:]
    modelname = filename[len('separation_wct_'):filename.find('_randomweights')]
    print('Read results for model ' + modelname)
    if modelname in dfdict:
        dfdict[modelname] = pd.concat([dfdict[modelname], df_])
    else:
        dfdict[modelname] = df_.copy()

In [ ]:
dfdict

In [ ]:
dfdict['manne'].info()

### Remove duplicated header rows from both dataframes 

In [ ]:
def find_invalid_values(df):
    all_invalid_values = set()
    for col in df:
        if col not in ['executionId','ub_name','instance_name','budget_Gamma','permutation','is_optimal','validated']:
            # 'alpha','n','m','cmax','time_spent','time_to_best_sol','iterations','num_visited_solutions','num_improvements','gap','lb','cost','cmax_dp'
            a = pd.to_numeric(df[col], errors='coerce')
            idx = a.isna()
            invalid_values = df.loc[idx][col].unique()
            all_invalid_values.update(invalid_values)
        #elif col in ['is_optimal','validated']
    print('Invalid values:', all_invalid_values)
    return all_invalid_values

In [ ]:
for key, df in dfdict.items():
    dfdict[key] = df[(df['executionId'] != 'executionId')]
    print(key, dfdict[key].dtypes)

### Convert column types from object 

In [ ]:
def convert_column_types(df):
    for col in df:
        if col in ['alpha','n','m','wct','budget_Gamma','time_spent','time_to_best_sol','iterations','num_visited_solutions','num_improvements','gap','lb','cost','wct_validation', 'mp_total_time', 'sp_total_time', 'seq']:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        elif col in ['is_optimal','validated']:
            df[col] = df[col].astype('bool')
    return df

In [ ]:
%%time
for key, df in dfdict.items():
    dfdict[key] = convert_column_types(df)
    print(key, dfdict[key].dtypes)

### Trim existing string columns 

In [ ]:
def trim_all_columns(df):
    """
    Trim whitespace from ends of each value across all series in dataframe
    """
    trim_strings = lambda x: x.strip() if isinstance(x, str) else x
    return df.applymap(trim_strings)

In [ ]:
for key, df in dfdict.items():
    dfdict[key] = trim_all_columns(df)
    print(key)

### Include a column with the name of the underlying C&CG MILP Model

In [ ]:
for key, df in dfdict.items():
    dfdict[key]['model'] = key

### Include a column with the name of the instance type (ying or tail)

In [ ]:
for key, df in dfdict.items():
    dfdict[key]['instance_type'] = df['instance_name'].apply(lambda x: 'tail' if (x[0:5] == 'tail') else 'ying')

### Concatenate dataframes 

In [ ]:
df = pd.concat(list(dfdict.values()))
df.head(4)

### Fix instance names 

The original instance names, in the instance file zip, were assembled incorrectly.

The problem lies in the alpha percentage. We are now going to fix this issue.

In [ ]:
df['seq'] = df['instance_name'].apply(lambda x: x[x.find('_')-2:x.find('_')])
df['n_str'] = df['n'].astype(str).str.zfill(3)
df['alpha_str'] = df['alpha'].astype(str)

In [ ]:
df['instance_name'] = 'RB' + df['n_str'] + df['alpha_str'] + df['seq'] + '_' + df['instance_name'].apply(lambda x: x[x.find('_')+1:])
df.drop(columns=['n_str', 'alpha_str'], inplace=True)
df.head(4)

### Round columns containing time (in seconds) 

In [ ]:
df['time_spent'] = df['time_spent'].round(2)
df['time_to_best_sol'] = df['time_to_best_sol'].round(2)
df['mp_total_time'] = df['mp_total_time'].round(2)
df['sp_total_time'] = df['sp_total_time'].round(2)

### Sort data according to model, instance_name, alpha, n, m and Gamma and set index

In [ ]:
print('Sorting dataset...')
df = df.sort_values(['model', 'n', 'm', 'alpha', 'seq', 'budget_Gamma', 'instance_type'])
display(df.dtypes)
df = df.set_index(['model', 'n', 'm', 'alpha', 'seq', 'budget_Gamma', 'instance_type'])
display(df.head(6))

### Find missing results, for a given value of alpha, n and m

For a given group of alpha, n, m and budget_Gamma, there should be 10 results.

First we will build a dataframe with the instances list and all required budget values.

In [ ]:
data = []
rootfolder = os.getcwd()
file_list = Path(os.path.join(rootfolder, 'instances', 'robust')).rglob('*.txt')
file_set = set()
for path in file_list:
    instance_path = path.name
    #print('instance_path: ' + instance_path)    
    if '.txt' not in instance_path:
        continue
    if 'tail' in instance_path:
        if instance_path[:instance_path.find('_')] not in ['tail001', 'tail002', 'tail003', 'tail004', 'tail005', 'tail006', 'tail007', 'tail008', 'tail009', 'tail010']:
            #print(instance_path[:instance_path.find('_')])
            continue
    instance_name = instance_path[instance_path.rfind(os.path.sep)+1:]
    file_set.add(instance_name)
print(file_set, file_set)
for instance_name in file_set:
    #print('instance_name: ' + instance_name)
    seq = instance_name[instance_name.find('_')-2:instance_name.find('_')]
    info = instance_name[instance_name.find('_')+1:]
    n = info[:info.find('_')]
    info = info[info.find('_')+1:]
    m = info[:info.find('_')]
    info = info[info.find('_')+1:]
    alpha = info[:info.find('_')]
    instance_type = 'ying'
    if 'tail' in instance_name:
        instance_type = 'tail'
    else:
        if int(n) > 20:
            continue
    for gamma in [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]:
        budget_gamma = int(gamma) * (int(m) * int(n)) / 100.0
        for model in list(dfdict.keys()):
            data.append([model, seq, alpha, int(n), int(m), budget_gamma, instance_type])
df_instances = pd.DataFrame(data, columns=['model', 'seq', 'alpha', 'n', 'm', 'budget_Gamma', 'instance_type'])
for col in df_instances:
    if col in ['alpha','n','m','budget_Gamma']:
        df_instances[col] = pd.to_numeric(df_instances[col], errors='coerce')
display(df_instances.dtypes)
df_instances = df_instances.set_index(['model', 'n', 'm', 'alpha', 'seq', 'budget_Gamma', 'instance_type'])
display(df_instances)

In [ ]:
df_instances.info()

In [ ]:
df.info()

Now, lets join the instances dataframe with the results one (left join).

In [ ]:
df_joined = df_instances.join(df, how='left')
df_joined

In [ ]:
df_joined.reset_index()[['model', 'n', 'm', 'alpha', 'seq', 'budget_Gamma', 'instance_type']]

Now we will export to CSV a list with all rows with NaN values (missing experimental results).

In [ ]:
missing_df = df_joined[df_joined.isnull().any(axis=1)].reset_index()[['model', 'n', 'm', 'alpha', 'seq', 'budget_Gamma', 'instance_type']]
outputfolder = os.path.join(os.getcwd(), 'results', 'consolidated')
if not os.path.exists(outputfolder):
    os.makedirs(outputfolder)
print('Saving file on folder: ' + outputfolder)
fname = os.path.join(outputfolder, 'RPFS_TWCT_missing_results.csv')
missing_df.to_csv(fname, sep=';')
print('Saved: ' + fname)

In [ ]:
df_grouped = df.groupby(['alpha', 'n', 'm', 'budget_Gamma']).agg({'executionId' : ['count']}).reset_index()
df_grouped.columns = [ ' '.join(str(i) for i in col) for col in df_grouped.columns]
#df_grouped.reset_index(inplace=True)
df_grouped

In [ ]:
table = pd.pivot_table(df, values='executionId', index=['alpha', 'n'], columns=['budget_Gamma'], aggfunc='count', fill_value=0)
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    display(table)

### Export the dataset to CSV file 

In [ ]:
%%time

outputfolder = os.path.join(os.getcwd(), 'results', 'consolidated')
print('Saving file on folder: ' + outputfolder)
fname = os.path.join(outputfolder, 'RPFS_TWCT_all_results.csv')
df.to_csv(fname, sep=';')
print('Saved: ' + fname)